# The Recording container

WIP

The class `cedalion.dataclasses.Recording` can be used to carry related data objects through the program. It can store time series, masks, auxiliary timeseries, probe, headmodel and stimulus information as well as meta data about the recording.
It has the following properties:

- It resembles the [NIRS group in the snirf specification](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsi), which provides storage for much of the data stored in a `Recording` (e.g. time series map to [data elements](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsidataj), [probe](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsiprobe), [stimulus](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsistimj) and [meta data](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsimetadatatags) are stored per NIRS element, etc). Consequently, the methods `cedalion.io.read_snirf` and `cedalion.io.write_snirf` methods operate on lists of recordings.
- different time series and masks are stored in ordered dictionaries
  - user differentiates time series by name
  - there is a set of canonical names used by `read_snirf` to assign names to time series
    ```
    CANONICAL_NAMES = {
          "unprocessed raw": "amp",
          "processed raw": "amp",
          "processed dOD": "od",
          "processed concentrations": "conc",
          "processed central moments": "moments",
          "processed blood flow index": "bfi",
          "processed HRF dOD": "hrf_od",
          "processed HRF central moments": "hrf_moments",
          "processed HRF concentrations": "hrf_conc",
          "processed HRF blood flow index": "hrf_bfi",
          "processed absorption coefficient": "mua",
          "processed scattering coefficient": "musp",
    }
    ```
- time series are stored in the dictionaries in the order that they were added
- convenient access to the last changed time series + canonical names -> consecutive transformations of time series without the need to specify time series by name -> workflows
- `rec[key]` is a shortcut for `rec.timeseries[key]` 
- not all information stored in a `Recording` can be stored in snirf files, e.g. for masks, the headmodel and auxiliar objects there is no provision in the snirf specification. We will probably use sidecard files or sidecar hdf groups to store these.